In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as pyplot
import os

In [3]:
X = np.load('optical_flows.npy')  # Load optical flows data
y = np.load('labels.npy')  # Load labels dat

In [4]:
X[0].shape

(224, 224, 2)

In [8]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16 # type: ignore
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, GlobalAveragePooling2D # type: ignore

# Define a custom input layer for 2 channels
input_tensor = Input(shape=(224, 224, 2))

# Load the VGG16 model without the top layers (use the functional API)
# We use input_tensor instead of the original input layer of VGG16
x = Conv2D(64, (3,3), activation='relu', padding='same', name='block_conv1')(input_tensor)
base_model = VGG16(weights='imagenet', include_top=False)

for layer in base_model.layers[2:]:
    x = layer(x)

# Add custom layers on top of the base model
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output_layer = Dense(4, activation='softmax')(x)  # 4 classes for emotions: 0, 1, 2, 3

# Create the new model
model = Model(inputs=input_tensor, outputs=output_layer)

# Freeze the layers in the base model
for layer in base_model.layers[1:]:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display model summary to verify the architecture
model.summary()




Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 2)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_conv1 (Conv2D)            │ (None, 224, 224, 64)   │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,780,292 (56.38 MB)

 Trainable params: 67,396 (263.27 KB)

 Non-trainable params: 14,712,896 (56.13 MB)

In [9]:
#Shuffle data
from sklearn.utils import shuffle # type: ignore

# Shuffle X and y
X, y = shuffle(X, y, random_state=42)

In [10]:
from sklearn.model_selection import train_test_split

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=16,
    verbose=1
)

Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 114s 4s/step - accuracy: 0.2581 - loss: 1.4055 - val_accuracy: 0.6458 - val_loss: 0.8655
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 105s 4s/step - accuracy: 0.6590 - loss: 0.8704 - val_accuracy: 0.8229 - val_loss: 0.5953
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 113s 5s/step - accuracy: 0.8516 - loss: 0.6304 - val_accuracy: 0.8333 - val_loss: 0.4795
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 114s 5s/step - accuracy: 0.8894 - loss: 0.4894 - val_accuracy: 0.8438 - val_loss: 0.4141
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 111s 5s/step - accuracy: 0.8902 - loss: 0.4075 - val_accuracy: 0.8750 - val_loss: 0.3779
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 109s 5s/step - accuracy: 0.9062 - loss: 0.3395 - val_accuracy: 0.8750 - val_loss: 0.3748
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 108s 5s/step - accuracy: 0.9199 - loss: 0.2739 - val_accuracy: 0.8750 - val_loss: 0.2974
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 118s 5s/step - accuracy: 0.8997 - loss: 0.2884 - val_accuracy: 0.9167 - v